In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv('./data/human_game_data.csv')
data2 = pd.read_csv('./data/RL_game_data.csv')

In [4]:
data1.describe()

,Elapsed Time,Score,Accuracy,Asteroids Hit
count,34.000000,34.000000,34.000000,34.000000
mean,64.123235,3064.000000,0.587353,31.941176
std,41.554926,1828.865852,0.196010,19.609740
min,21.200000,0.000000,0.000000,0.000000
25%,39.577500,1850.000000,0.465000,19.000000
50%,50.870000,2716.000000,0.605000,28.000000
75%,70.282500,3654.000000,0.722500,38.000000
max,207.810000,10130.000000,1.000000,110.000000


In [5]:
data2.describe()

,Elapsed Time,Score,Accuracy,Asteroids Hit
count,57.000000,57.000000,57.00000,57.000000
mean,78.209298,3491.964912,0.44193,36.596491
std,143.827916,611.893255,0.06621,6.315682
min,37.650000,2368.000000,0.33000,25.000000
25%,48.720000,3052.000000,0.39000,32.000000
50%,58.470000,3430.000000,0.43000,36.000000
75%,65.030000,3798.000000,0.48000,40.000000
max,1132.440000,5018.000000,0.65000,52.000000


In [6]:
data1

,Game Date,Game Time,Elapsed Time,Reason,Score,Accuracy,Asteroids Hit
0,2024-06-07,10:39:11,21.20,Player Quit,1656,0.80,17
1,2024-06-07,10:40:28,26.30,Spacecraft Health 0,2126,0.59,22
2,2024-06-07,10:43:18,47.03,Player Quit,2244,0.82,23
3,2024-06-07,11:40:40,23.59,Player Quit,1466,0.88,15
4,2024-06-07,11:41:31,29.39,Spacecraft Health 0,0,0.00,0
5,2024-06-07,18:18:40,42.31,Spacecraft Health 0,776,0.67,8
6,2024-06-07,18:19:53,36.33,Player Quit,1556,0.73,16
7,2024-06-10,14:47:38,38.80,Player Quit,1666,1.00,17
8,2024-06-11,11:57:45,53.05,Spacecraft Health 0,1558,0.76,16
9,2024-06-11,12:02:32,27.11,Player Quit,2430,0.31,26


In [9]:
data1['Reason'].value_counts()

Reason
Spacecraft Health 0    14
Spacecraft Fuel 0      13
Player Quit             7
Name: count, dtype: int64

In [10]:
data2['Reason'].value_counts()

Reason
Spacecraft Health 0    51
Spacecraft Fuel 0       6
Name: count, dtype: int64